In [ ]:
map_rels_no = {'COM':0, 'CONTR':1, 'CORR':2, 'QAP':3, 'ACK':4,'ELAB':5,
                 'CLARIFQ':6, 'COND':7, 'CONTIN':8, 'RES':9, 'EXPL':10, 'QELAB':11,
                 'ALT':12, 'NARR':13, 'CONFQ':14, 'SEQ':15, 'NULL':16}

reverse_relations = {0:'COM', 1: 'CONTR', 2 :'CORR', 3: 'QAP', 4: 'ACK', 5: 'ELAB',
            6: 'CLARIFQ', 7: 'COND', 8: 'CONTIN', 9: 'RES', 10: 'EXPL',
            11: 'QELAB', 12: 'ALT', 13: 'NARR', 14: 'CONFQ', 15: 'SEQ', 16: 'NULL'}

# reverse_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
#                  'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
#                  'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}


In [1]:
map_rels_str = {'Comment':'COM', 'Contrast':'CONTR', 'Correction':'CORR', 'Question-answer_pair':'QAP', 'Acknowledgement':'ACK','Elaboration':'ELAB',
                 'Clarification_question':'CLARIFQ', 'Conditional':'COND', 'Continuation':'CONTIN', 'Result':'RES', 'Explanation':'EXPL', 'Q-Elab':'QELAB',
                 'Alternation':'ALT', 'Narration':'NARR', 'Background':'BACK', 'Parallel':'PAR', 'Sequence':'SEQ', 'Question_answer_pair':'QAP',  
                 'Q_Elab':'QELAB', 'Confirmation_question' : 'CONFQ'}


## STAC counts

In [3]:
#STAC COUNTS real quick
import json
from collections import Counter, defaultdict
import jsonlines
import pickle
home=%pwd
mc_path = '/home/kate/minecraft_utils/llm_annotator/annotated_data/TEST_101_bert.json'
local_path = '/home/kate/minecraft_utils/llm_annotator/local_model_flat_test.jsonl'
local_flat_path = '/home/kate/minecraft_utils/llm_annotator/local_model_flat_train.jsonl'
mc_check = '/home/kate/LREC/data/TEST_101_bert.json'
llama_parse = '/home/kate/minecraft_utils/llm_annotator/parser_test_moves_15.jsonl'
bert_data = '/home/kate/minecraft_utils/parsing_scores/msdc_bert/full_scores_multi_d10.pkl'

In [ ]:
with open(bert_data, 'rb') as f:
    test_multi = pickle.load(f)

In [ ]:
len(test_multi)

In [ ]:
bert_test = []
for t in test_multi:
    bert_test.append(reverse_relations[t[3]])

In [ ]:
Counter(bert_test)

In [ ]:
Counter(bert_test)

In [ ]:
with open(mc_check, 'r') as j:
    jfile = json.load(j)
    data = jfile

In [ ]:
#check relation counts
rels = []
for d in data:
    rels.extend([d['type'] for d in d['relations'] if d['x'] < d['y'] and abs(d['y'] - d['x']) <=10])

In [ ]:
Counter(rels)

In [4]:
# local = []
# with jsonlines.open(local_path) as reader:
#     for i, obj in enumerate(reader):
#         if i%2 == 0:
#             s = obj['sample'].split('\n')
#             out = (i, obj['PS'], s[0], s[1])
#             local.append(out)

local = []
with jsonlines.open(local_path) as reader:
    for i, obj in enumerate(reader):
        if obj['PS'] != 'NONE':
            local.append(obj['PS'])

In [5]:
Counter(local)

Counter({'QAP': 305,
         'COM': 164,
         'ACK': 147,
         'CONTIN': 112,
         'ELAB': 101,
         'QELAB': 72,
         'CONTR': 43,
         'CLARIFQ': 33,
         'EXPL': 30,
         'RES': 28,
         'CORR': 21,
         'ALT': 19,
         'PAR': 15,
         'COND': 11,
         'NARR': 9})

In [58]:
len(local)

10878

In [ ]:
local[:2]

In [ ]:
#check RES
loc_res = []
for loc in local:
    if loc[1] == 'RES':
        edu_one_ind = loc[2].split('<')[0].strip()
        edu_one_text = loc[2].split('>')[1].strip()
        edu_two_ind = loc[3].split('<')[0].strip()
        edu_two_text = loc[3].split('>')[1].strip()
        loc_res.append((loc[0], edu_one_ind, edu_one_text, edu_two_ind, edu_two_text))

In [ ]:
print(loc_res[0])

#### backwards relations

In [ ]:
bkwds = defaultdict(list)
for d in data:
    for rel in d['relations']:
        if rel['y'] < rel['x']:
            bkwds[map_rels_str[rel['type']]].append(rel['y'] - rel['x'])

In [ ]:
for k in bkwds.keys():
    print('{} rel : {}'.format(k, len(bkwds[k])))

STAC SQUISHED
COND rel : 68
COM rel : 57
EXPL rel : 4
BACK rel : 10
ELAB rel : 3

STAC LINGUISTIC
COND rel : 67
CONTIN rel : 1
COM rel : 61
BACK rel : 8
ELAB rel : 3
EXPL rel : 8

In [ ]:
edus = 0
eeus = 0
for d in stac['dialogues']:
    total_dus = len(d['edus'])
    eeu_count = len([e for e in d['edus'] if e['speaker'] in ['UI', 'Server']])
    eeus += eeu_count
    edu_count = total_dus - eeu_count
    edus += edu_count

print("edus: ", edus)
print("eeus: ", eeus)

In [ ]:
cdus = 0
for d in stac['dialogues']:
    cdus += len(d['cdus'])

print("cdus: ", cdus)


In [ ]:
rels = 0
for d in stac['dialogues']:
    rels += len(d['relations'])

print("rels: ", rels)

In [ ]:
mpdu = 0
for d in stac['dialogues']:
    
    rels_count = Counter([g['target'] for g in d['relations']])
    mps = [r[0] for r in rels_count.items() if r[1] > 1] 
    mpdu += len(mps)

print("mpdus: ", mpdu)

In [ ]:
# #these are the STAC relations
# map_relations = {'COM': 0, 'CONT': 1, 'CORR': 2, 'QAP': 3, 'PAR': 4, 'ACK': 5,
#             'ELAB': 6, 'CLARIFQ': 7, 'COND': 8, 'CONTIN': 9, 'RES': 10, 'EXPL': 11,
#             'QELAB': 12, 'ALT': 13, 'NARR': 14, 'BACK': 15, 'NULL': 16, 'SEQ' : 17}

# reverse_relations = {0:'COM', 1: 'CONT', 2 :'CORR', 3: 'QAP', 4: 'PAR', 5: 'ACK',
#             6: 'ELAB', 7: 'CLARIFQ', 8: 'COND', 9: 'CONT', 10: 'RES', 11: 'EXPL',
#             12: 'QELAB', 13: 'ALT', 14: 'NARR', 15: 'BACK', 16: 'NULL', 17: 'SEQ'}

In [ ]:
# labels = ['COM','CONTR','CORR','QAP','ACK','ELAB','CLARIFQ','COND','CONTIN',
#               'RES','EXPL','QELAB','ALT','NARR','CONFQ','SEQ','NULL']

# labels = ['COM', 'CONT', 'CORR', 'QAP', 'PAR', 'ACK',
#             'ELAB', 'CLARIFQ', 'COND', 'CONT', 'RES', 'EXPL',
#             'QELAB', 'ALT', 'NARR', 'BACK', 'NULL', 'SEQ']

## BERTLine scores

In [ ]:
import pickle
import pandas
import numpy as np
from collections import Counter, defaultdict

#### Narration counts

In [ ]:
home=%pwd
picklepath = home + '/msdc_bert/'

In [ ]:
with open(picklepath + 'full_scores_multi_d10_NEWTEST.pkl', 'rb') as f:
    test_multi = pickle.load(f)

In [ ]:
test_multi[:2]

In [ ]:
gold_distances = []
tp_distances = []
fp_distances = []
fn_distances = []

In [ ]:
for t in test_multi:
    if t[3] == 13 and t[4] == 13:
        tp_distances.append(t[2] - t[1])
    elif t[3] == 13 and t[4] != 13:
        fn_distances.append(t[2] - t[1])
    elif t[3] != 13 and t[4] == 13:
        fp_distances.append(t[2] - t[1])
    
    if t[3] == 13:
        gold_distances.append(t[2] - t[1])
    

In [ ]:
tp_counts = Counter(tp_distances)
fp_counts = Counter(fp_distances)
fn_counts = Counter(fn_distances)
gold_counts = Counter(gold_distances)

In [ ]:
labels = [d for d in range(1,16)]
head = ['gold', 'tp', 'fn', 'fp', 'F1']
data = [] #a list of lists

In [ ]:
for d in labels:
    row = []
    for count_list in [gold_counts, tp_counts, fn_counts, fp_counts]:
        num = [c[1] for c in count_list.items() if c[0] == d]
        if len(num) > 0:
            row.append(num[0])
        else:
            row.append(0)
        #calculate distance F1
    if row[1] != 0:
        microf1 = round(row[1]/(row[1] + 0.5*(row[3] + row[2])), 2)
    else:
        microf1 = 0.0
    row.append(microf1)

    data.append(row)

In [ ]:
print('Bertline Narration predictions')
print('                                         ')
print(pandas.DataFrame(data, labels, head))
print('                                          ')

#### END Narration counts

In [ ]:
home=%pwd
picklepath = home + '/msdc_bert/'
# picklepath = home + '/stac_bert/'

In [ ]:
# with open(picklepath + 'scores_linear_d10_newtest.pkl', 'rb') as f:
#     test_lin = pickle.load(f)

with open(picklepath + 'full_scores_multi_d10_NEWTEST.pkl', 'rb') as f:
    test_multi = pickle.load(f)

# with open(picklepath + 'comparison_multi_stac.pkl', 'rb') as f:
#     test_multi = pickle.load(f)

# with open(picklepath + 'linear_preds_stac.pkl', 'rb') as f:
#     test_lin = pickle.load(f)

# with open(picklepath + 'comparison_multi_stac_connected.pkl', 'rb') as f:
#     test_multi = pickle.load(f)

# with open(picklepath + 'linear_preds_stac_connect.pkl', 'rb') as f:
#     test_lin = pickle.load(f)

In [ ]:
test_multi[:10]

In [ ]:
len(test_multi)

In [ ]:
#check that every edu is attached somewhere 
test_multi[-1]

In [ ]:
graph_dict_preds = defaultdict(list)
graph_dict_gold = defaultdict(list)
for t in test_multi:
    if t[4] != 16:
        graph_dict_preds[t[0]].append(t[2])
    if t[3] != 16:
        graph_dict_gold[t[0]].append(t[2])

In [ ]:
missing_links = 0
for r in range(100):
    p = graph_dict_preds[r]
    g = graph_dict_gold[r]
    #assert len(p) <= len(g), 'at index {}'.format(r) #make sure the preds are contained in the gold
    missing = len(set(g) - set(p))
    missing_links += missing


In [ ]:
print(missing_links)

In [ ]:
graph_dict_preds[2]

In [ ]:
graph_dict_gold[2]

In [ ]:
test_lin[:10]

In [ ]:
len([t for t in test_lin if t[5] == 1 or t[3] == 1])

In [ ]:
true_attach = [t[3] for t in test_lin]
attach_preds = [t[5] for t in test_lin]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, classification_report, ConfusionMatrixDisplay, confusion_matrix

In [ ]:
prf = precision_recall_fscore_support(true_attach, attach_preds, average='binary')
print("Attachment F1:", prf[2])
print("Attachment Average Precision:", prf[0])
print("Attachment Average Recall:", prf[1])


In [ ]:
true_labels = [reverse_relations[t[3]] for t in test_multi]
labels_pred = [reverse_relations[t[4]] for t in test_multi]

In [ ]:
print(classification_report(true_labels, labels_pred))

In [ ]:
f = open(home + "/bert_matrix.txt","w")
print(classification_report(true_labels, labels_pred), file=f)
f.close()

In [ ]:
gold_list = [t[3] for t in test_multi]
pred_list = [t[4] for t in test_multi]

In [ ]:
d = classification_report(gold_list,pred_list,target_names=labels,output_dict=True)
prec = 0
rec = 0
f1 = 0 
count = 0

for label in labels:
    if label!="NULL":
        prec+=d[label]["precision"]*d[label]["support"]
        rec+=d[label]["recall"]*d[label]["support"]
        f1+=d[label]["f1-score"]*d[label]["support"]
        count+=d[label]["support"]
        # checking that support is same as the number of ground truth instance for the label
        #assert d[label]["support"] == Counter(g_label_l)[label]
        


print("Weighted Average Precision:", prec/count)
print("Weighted Average Recall:", rec/count)
print("Weighted Average F1 score:", f1/count)

#change to numbers 
'COM':0, 
'CONTR':1, 
'CORR':2, 
'QAP':3, 
'ACK':4,
'ELAB':5,
'CLARIFQ':6, 
'COND':7, 
'CONTIN':8, 
'RES':9, 
'EXPL':10, 
'QELAB':11,
'ALT':12, 
'NARR':13, 
'CONFQ':14, 'SEQ':15, 'NULL':16

In [ ]:
correct = [map_rels_no[t] for t in corr_]
predicted = [map_rels_no[t] for t in pred_]

In [ ]:
cm = confusion_matrix(correct,predicted)
ConfusionMatrixDisplay(cm).plot()

In [ ]:
with open(picklepath + 'tp_dict.pkl', 'rb') as f:
    tp_dict = pickle.load(f)

with open(picklepath + 'fn_dict.pkl', 'rb') as f:
    fn_dict = pickle.load(f)

with open(picklepath + 'fp_dict.pkl', 'rb') as f:
    fp_dict = pickle.load(f)

In [ ]:
for key in sorted(tp_dict.keys()):
    print(key, len(tp_dict[key]))

In [ ]:
Counter(tp_dict[10])

In [ ]:
Counter(tp_dict[2])

In [ ]:
for key in sorted(fn_dict.keys()):
    print(key, len(fn_dict[key]))

## multiparent edus

In [ ]:
from collections import Counter
import json

In [ ]:
home=%pwd
gold_data_path = '/home/kate/minecraft_utils/llm_annotator/annotated_data/TEST_101_bert.json'

In [ ]:
with open(gold_data_path, 'r') as j:
    jfile = json.load(j)
    gold = jfile

In [ ]:
len(gold)

In [ ]:
test_multi[:2]

In [ ]:
gold_count = [] #these are the ones that have double endpoints in common
pred_count = []
type_compare = [] #[i, x, y, gold_rel, pred_re] AND source compare

for i, game in enumerate(gold[:1]):
    rels_count = Counter([g['y'] for g in game['relations']])
    count = [r[0] for r in rels_count.items() if r[1] > 1]
    #print(count)
    mp_list = []
    for r in gold[i]['relations']:
        if r['y'] in count:
            mp_list.append([i, r['x'], r['y'], reverse_relations[r['type']]])
    # for m in mp_list:
    #     print(m)
    

In [ ]:
mp_list[:1]

In [ ]:
total_mpedus_gold = 0
total_mpedus_pred = 0
total_overlap_target = []
total_target_accuracy = []

for i, game in enumerate(gold):
    #get predictions for this game -- all predicted endpoints
    preds = [e for e in test_multi if e[0] == i and e[4] != 16]
    rels_count = Counter([g['y'] for g in game['relations']])
    gold_count = [r[0] for r in rels_count.items() if r[1] > 1]
    total_mpedus_gold += len(gold_count)
    preds_counter = Counter([p[2] for p in preds])
    preds_count = [r[0] for r in preds_counter.items() if r[1] > 1] 
    total_mpedus_pred += len(preds_count)
    #get matching targets in pred
    # print("gold multi: ", len(gold_count))
    # print("pred multi: ", len(preds_count))
    overlap = [m for m in preds_count if m in gold_count]
    # print('overlap: ', len(overlap))
    # print(overlap)
    # print('-------------------------------')
    total_overlap_target.append(len(overlap)/len(gold_count)) #save total overlap
    #total_target_accuracy = []
    #now for each target in overlap, see if the source is correct
    for target in overlap:
        gold_sources = [gr['x'] for gr in gold[i]['relations'] if gr['y'] == target]
        preds_sources = [pr[1] for pr in preds if pr[2] == target]
        accuracy = len([n for n in preds_sources if n in gold_sources])/len(gold_sources)
        total_target_accuracy.append(accuracy)

        # print('target: {} '.format(target))
        # print('gold sources: {}'.format(gold_sources))
        # print('pred sources: {} '.format(preds_sources))
        # print('accuracy : {}'.format(accuracy))
        # print('---------------------------------')


#now for each target in overlap, see if target is correct 
print('total mpdus in gold: {}'.format(total_mpedus_gold))
print('total mpdus predicted: {}'.format(total_mpedus_pred))
print('mean target accuracy: {}'.format(np.mean(total_overlap_target)))
print('mean source accuracy: {}'.format(np.mean(total_target_accuracy)))

##so we got about 45 percent of the multiparent edus correct, and of those, we got 87 percent of the sources correct 

